<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/main/Dataviter_WS_del_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop sesjon 4: Utforsking og visualisering



Hittil har vi sett på hvordan vi laster inn og tranformerer data i Google-plattformen og hvordan vi kan bruke BigQuery til å hente ut de dataene vi er interessert i og gjøre enkle analyser og aggregeringer. Nå skal vi se på et lett tilgjengelige verktøy for å utforske dataene videre og visualisere funnene våre - nemlig Data Studio.



Overordnet plan

*   Finne partisjonert tabell fra del 3 i GUI
*   Explore With Data Studio - bli kjent med GUI og sette opp linjediagram med sum målinger og antall målestasjoner over tid
*   Se etter interessante trekk ved tidsserien
*   --> Vi må sammenlikne data fra de samme stasjonene. Definer ny spørring som henter unike stasjoner 1. januar 2017 inner joiner med unike stasjoner 31. desember 2019 (dermed kun stasjoner som var med i begge perioder)






## Utforsking i Data Studio

Vi tar utgangspunkt i den tidspartisjonerte tabellen vi opprettet i forrige økt. Denne gir oss en rad for hver kombinasjon av målepunktsidentifikator (```trpId```) og tidspunkt (```startOfHour``` og ```lokaltid```) med informasjon om antall kjøretøy som er målt (```volume```) og hvilken kommune målepunktet ligger i (```kommunenavn```).

Om du ikke har opprettet denne tabellen allerede, kan du kopiere og kjøre spørringen under. Husk å angi ditt eget prosjekt etter ```CREATE TABLE```.

In [ ]:
%%bigquery --project $project
CREATE TABLE `saga-workshop-data-vu8x.workshop.produksjonstabell` PARTITION BY DATE(startOfHour)
AS (
SELECT
  a.trpId,
  a.startOfHour,
  DATETIME(a.startOfHour,"Europe/Oslo") lokaltid,
  a.total.volumeNumbers.volume,
  c.kommunenavn,
  c.geometry
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata_v2` a
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` b
ON
  a.trpId = b.Id
JOIN
  `saga-workshop-data-vu8x.workshop.kommuner` c
ON
  ST_WITHIN( ST_GEOGPOINT(b.location.coordinates.latLon.lon, b.location.coordinates.latLon.lat), c.geometry)
)

Vi skal nå utforske disse dataene og se om vi kan finne noen påfallende mønstre eller andre ting vi kan undersøke nærmere. Til det bruker vi verktøyet Data Studio, som er tett integrert med BigQuerys grafiske grensesnitt:


1.   Velg den tidspartisjonerte tabellen i Explorer-menyen til venstre
2.   Klikk "Export" oppe til høyre
3.   Velg "Explore with Data Studio" i nedtrekksmenyen

Du skal nå få opp en ny fane i nettleseren med et nytt Data Studio-dokument. Foreløpig inneholder det bare en tabellvisning, men vi kan velge en rekke ulike diagrammer øverst i menyen til høyre. Her mistenker vi å kunne finne noe interessant ved å se på variasjon over tid, så vi velger *diagram over tidsrekker*.

Under diagramvelgeren har vi to faner med innstillinger - en for å velge hvilke data vi vil vise og en for å formatere farger, akser og den slags. I datafanen er dette det viktigste vi må forholde oss til:


*   **Dimensjon**. Her angir vi hvilket felt vi vil knytte til x-aksen og med hvilken oppløsning. ```startOfHour```-feltet er allerede valgt for oss, så dette behøver vi ikke å endre på.
*   **Fordelingsdimensjon**. Her kan vi angi et felt som inneholder ulike kategorier dersom vi ønsker å vise hver kategori som en egen serie i diagrammet. Her har Data Studio valgt å vise hvert målepunkt som en egen serie, men det ønsker vi jo ikke. Vi fjerner derfor hele inndelingen i kategorier ved å sveve over feltnavnet og trykke på x-en som dukker opp. 
*   **Beregning**. Her knytter vi felter til y-aksen(e). Nå viser DataStudio bare ```Record Count```, altså antall rader tabellen har for hver dato. Vi klikker på feltnavnet og velger heller ```volume```. DataStudio skal nå velge å vise summen av ```volume```-feltene for hver dato, men vi kunne like gjerne valgt å vise gjennomsnitt eller antall rader ved å klikke på "SUM"-ikonet ved siden av feltnavnet.

Nå har vi en tidsserie som viser utviklingen i antall målte kjøretøy per døgn.

<br>

**Diskusjon**: Er det noen påfallende utviklingstrekk, mønstre eller andre ting vi kan legge merke til her?




### Diskusjonspunkter



Det er en åpenbar **voksende trend** her - vi går fra null trafikk i 2014 til 20 millioner passeringer i 2019.
  * Hypotese: Det blir ikke mer trafikk, men flere *målinger*.
  * Vi undersøker dette ved å legge til en ny serie for antall målepunkter: Klikk og dra inn ```trpId```som nytt felt under *Beregning* -> gå til *Stil*-fanen og legg serie 2 til høyre akse
  * Hypotesen ser ut til å stemme - vi måler mer trafikk i takt med at det installeres flere tellepunkter.

Det er påfallende **regelmessighet i variasjonen** både på kort og mellomlang sikt.
  * Vi undersøker dette nærmere ved å se på kortere tidsperiode i større detalj.
  * Under *Standard dataperiode*, velg *Egendefinert* -> Bruk kalendergrensesnittet til å velge perioden fra 1. januar 2018 til 31. desember 2019.
  * Vi kan tydelig se effekten av ukedager, sommerferie, jul og påske.

Konklusjon: Det kan være interessant å undersøke disse systematiske variasjonene nærmere, men vi bør sammenligne de *samme* målepunktene over tid.


## Henting av et konsistent datasett

Vi ønsker å sammenligne data fra de *samme* målestasjonene over tid. Vi henter derfor en liste over alle målestasjonene og bruker ```GROUP BY```for å få én rad per målepunkt. Vi ønsker å se på data for 2018 til 2019 og angir dette med ```WHERE```.

Så bruker vi ```COUNT``` til å telle antall dager der vi har minst én måling fra et bestemt målepunkt. Dette kombinerer vi med ```HAVING```for å kun velge de målepunktene som har levert målinger i 730 dager, altså hver eneste dag i løpet av toårsperioden.

In [ ]:
%%bigquery --project $project
SELECT
    trpId
FROM
    `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE
    lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND volume > 0
GROUP BY trpId
HAVING COUNT(DISTINCT DATE(lokaltid)) >= 730

Denne spørringen returnerer en liste med alle målestasjoner som leverte minst én måling hver dag i den perioden vi ser på. Vi vil nå ha med alle feltene i datasetet, men bare for disse målepunktene:

In [ ]:
%%bigquery --project $project
SELECT
    *
FROM `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND trpId IN (
SELECT
    trpId
FROM
    `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE
    lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND volume > 0
GROUP BY trpId
HAVING COUNT(DISTINCT DATE(lokaltid)) >= 730
)

Vi kjører denne spørringen og utforsker resultatet i Data Studio - denne gangen ved å klike "Explore Data" fra rett over resultatet fra spørringen.

## Analyse av tidsvariasjon

Vi gjentar raskt stegene over med de nye dataene og ser at både ```volume``` og antall tellepunkter ser riktig ut. Nå skal vi bruke Data Studios innebygde funksjonalitet for tidsaggregering til å analysere de systematiske variasjonene over tid. Vi begynner med å se på systematiske forskjeller mellom ukedagene.

Under *Dimensjon* har vi nå valgt dato som tidsoppløsning. Dette kan vi endre ved å klikke på kalenderikonet ved siden av feltnavnet og sette det til *Ukedag*. I *Stil*-fanen setter vi y-aksen til å gå fra 0 og velger søylevisning. Vi har nå et diagram over totalt antall målte kjøretøy per ukedag gjennom de to årene vi ser på.

Enheten på y-aksen er nå i absolutte tall - men totalt antall målte kjøretøy for et vilkårlige sett med målepunkter gir ikke god mening å fortolke. Vi vil i stedet se hvilken *andel* av ukas totale trafikk som faller på hver ukedager. Det løser vi slik:
* Finne *Beregninger*-seksjonen i *Data*-fanen og klikke på SUM-ikonet ved siden av feltnavnet.
* Under Sammenligningsberegning, velg *Prosent av det totale antallet* og *I forhold til tilsvarende data*

Vi ser nå ukedagsvariasjonen for alle målepunkter i Norge under ett. Vi kan nå utforske forskjeller mellom kommunene ved å legge til et filter:
*   Finn ```kommunenavn``` i oversikten over tilgjengelige felt til høyre på skjermen
*   Dra feltet til *Filter*-boksen øverst på skjermen.
* Klikk på filteret, finn den kommunen du er interessert i og trykk på *Bare* og *Bruk*

Vi skal nå ha noe slikt som dette:

<img src='https://drive.google.com/uc?id=1e5PvCG-Ib8WKbwq-ORx_sodALoF94Dl9' width="800">


Med disse instillingene på plass er det fort gjort å analysere andre typer tidsvariasjon. Ved å justere oppløsningen under *Dimensjon* kan vi for eksempel se på variasjon mellom måneder, ukenummer eller klokkeslett.

## Egendefinerte felter

Å bruke filter til å vise mer enn én kommune fungerer fint hvis 

## Oppgave



1.   Vis med en figur hvordan trafikken fordeler seg gjennom året for Norge totalt sett
2.   Legg til et filter og finn enkeltkommuner som skiller seg fra de nasjonale tallene
3. Bruk egendefinerte felter for å sammenligne den relative fordelignen for flere kommuner i én og samme figur
4. Endre tidsoppløsningen og sammenlign de samme kommunene med tanke på døgn- og ukedagsvariasjon



## Løsningsforslag